In [ ]:
import cv2
from tqdm import tqdm
import os

# Function to process videos and save augmented images
def process_videos(path, subdir, subsubdir):
    output_dir = os.path.join(output, subdir,subsubdir)
    os.makedirs(output_dir, exist_ok=True)                   
    for video_file in tqdm(os.listdir(path), desc=f"Processing {subdir} {subsubdir} videos", unit = 'vid'):
        cap = cv2.VideoCapture(os.path.join(path, video_file))
        if not cap.isOpened():
            print(f"Failed to open video file {video_file}. Skipping...")
            continue

        frame_count = 0
        video_name = os.path.basename(video_file)[:-4]
        while cap.isOpened():
            success, image = cap.read()
            if not success:
#                 print(f"Finished processing {video_file} from {subdir} set.")
                break

            # Convert the BGR image to RGB.
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            # Process the image and find pose landmarks.
            output_path = os.path.join(output_dir, f"{video_name}_frame_{frame_count}.jpg")
#             print(output_path)
            cv2.imwrite(output_path, image)
            frame_count += 1
        cap.release()

    print(f"Finished processing all videos in {subdir} {subsubdir} videos")


# Process videos from each split with data augmentation
source_dir = './dataset'
output = './dest'
for sub in os.listdir(source_dir):
    srcsub = os.path.join(source_dir, sub)
#     os.makedirs(output_sub, exist_ok = True)
    for subsub in os.listdir(srcsub):
        srcsubsub = os.path.join(srcsub, subsub)
        process_videos(srcsubsub, sub, subsub)
print("Finished processing all videos.")


In [ ]:

import ultralytics
from ultralytics import YOLO
import os
os.environ['WANDB_MODE'] = 'disabled'
# Load a model
# model = YOLO("yolov8n.pt")  # load an official model
model = YOLO("yolov8n-cls.pt")  # load a custom model

# Validate the model
results = model.train(data="./dest", epochs=40, device = [0, 1],  imgsz=224, batch = 512) # no arguments needed, dataset and settings remembered

metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.top1  # top1 accuracy
metrics.top5  # top5 accuracy